In [1]:
from db_query import *
import os 
import json
ROOT_DIR = os.getcwd()

In [2]:
records = fetch_all_records({"version": LATEST})
results = list(records)
len(results)

720

In [3]:
for result in results:
    result['_id'] = str(result['_id'])

In [4]:

with open(ROOT_DIR + "/logs/results.json", "w") as f:
    json.dump(results, f, indent=4, default=str)

In [25]:
kdd99_results = [{"model": result["model_name"], 
                  "layers": str(result["encoder_param"]["layers"]),
                  "score": result["result"]["u2r"]["f1-score"]} for result in results if result['dataset']["name"] == 'kdd99' and not result["dataset"]["dropped"]]
kdd99d_results = [{"model": result["model_name"], 
                  "layers": str(result["encoder_param"]["layers"]),
                  "score": result["result"]["u2r"]["f1-score"]} for result in results if result['dataset']["name"] == 'kdd99' and result["dataset"]["dropped"]]


In [26]:
kdd99_df = pd.DataFrame(kdd99_results)
# 行をmodel, 列をlayersにする
kdd99_df.pivot(index="model", columns="layers", values="score").to_csv(ROOT_DIR + "/logs/kdd99_v1.4.0.csv")
kdd99d_df = pd.DataFrame(kdd99d_results)
# 行をmodel, 列をlayersにする
kdd99d_df.pivot(index="model", columns="layers", values="score").to_csv(ROOT_DIR + "/logs/kdd99d_v1.4.0.csv")

In [37]:
aggregated_results = [{
    "model": result["model_name"], 
    "dataset": result["dataset"]["name"],
    "layers": str(result["encoder_param"]["layers"]),
    "score": result["result"]["macro avg"]["f1-score"]} for result in results if result['dataset']["name"] not in ['kdd99', 'creditcardfraud']]

In [38]:
aggregated_df = pd.DataFrame(aggregated_results)

In [41]:
# 
aggregated_df.to_csv(ROOT_DIR + "/logs/aggregated_v1.4.0.csv")

In [27]:
creditcardfraud_results = [{
    "model": result["model_name"], 
    "layers": str(result["encoder_param"]["layers"]),
    "score": result["result"]["macro avg"]["f1-score"]} for result in results if result['dataset']["name"] == 'creditcardfraud']

In [29]:
creditcardfraud_df = pd.DataFrame(creditcardfraud_results)

In [32]:
creditcardfraud_df.sort_values(by="score", ascending=False).to_csv(ROOT_DIR + "/logs/creditcardfraud_v1.4.0.csv")

In [39]:
# CSV集計
DATASET = "kdd99"
li = []
importances = []
importances_idx = []
for result in results:
    if result['dataset']['ae_used_data'] == 'all':
        li.append({
        "id": result['_id'],
        "version": result['version'],
        "datetime": result['datetime'],
        "elapsed_time": result['elapsed_time'],
        "dataset": result['dataset']['name'] + "_d" if result['dataset'].get("dropped") == True else result['dataset']['name'],
        "ae_used_data": result['dataset']['ae_used_data'],
        "feature_num": result['dataset']['total_feature'],
        "model": result['model_name'],
        "layers": result['encoder_param']['layers'],
        **{k: result['result'][k]["f1-score"] for k in result['result']},
        })
        if result.get('importances') is not None:
            total = sum(list(result['importances'].values()))
            if total == 0:
                continue
            importance = {k: v / total for k, v in result['importances'].items()}
            importances.append(importance)
            importances_idx.append(li[-1]['dataset'] + "_" + li[-1]['model'] + "_" + li[-1]['ae_used_data'])
aggregate_df = pd.DataFrame(li)
importances_df = pd.DataFrame(importances, index=importances_idx)
aggregate_df.head()

,id,version,datetime,elapsed_time,dataset,ae_used_data,feature_num,model,layers,majority,minority,macro avg,normal,anomaly,dos,probe,r2l,u2r
0,6563ce9f82ebed258fc0d718,1.3.0,2023-11-26 23:02:55.389,0:00:34.143996,arrhythmia,all,283,LightGBM+optuna,"[20, 10, 5]",0.9781,0.4697,0.7239,NaN,NaN,NaN,NaN,NaN,NaN
1,6563cead628cf932b7e90c01,1.3.0,2023-11-26 23:03:09.127,0:00:47.863997,oil,all,54,LightGBM+optuna,"[20, 10, 5]",0.9823,0.4828,0.7326,NaN,NaN,NaN,NaN,NaN,NaN
2,6563ceb1b0ad2234356e3f36,1.3.0,2023-11-26 23:03:13.535,0:00:52.271996,solar_flare_m0,all,37,LightGBM+optuna,"[20, 10, 5]",0.9745,0.0000,0.4873,NaN,NaN,NaN,NaN,NaN,NaN
3,6563ceb2ae8cef214cb815d7,1.3.0,2023-11-26 23:03:14.306,0:00:53.042998,car_eval_4,all,26,LightGBM+optuna,"[20, 10, 5]",0.9952,0.8651,0.9301,NaN,NaN,NaN,NaN,NaN,NaN
4,6563ced1b0ad2234356e3f38,1.3.0,2023-11-26 23:03:44.636,0:00:30.535002,yeast_me2,all,13,LightGBM+optuna,"[20, 10, 5]",0.9803,0.1846,0.5825,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
importances_df.head()

,0,1,2,3,4,5,6,7,8,9,...,612,613,614,615,616,ae_5,ae_6,ae_7,ae_8,ae_9
arrhythmia_LightGBM+optuna_all,0.008487,0.000000,0.002829,0.004243,0.004243,0.004243,0.028289,0.025460,0.002829,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
oil_LightGBM+optuna_all,0.080148,0.015091,0.047284,0.045272,0.036553,0.015091,0.019450,0.041247,0.007378,0.027834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
solar_flare_m0_LightGBM+optuna_all,0.004939,0.002615,0.018594,0.007844,0.001743,0.003777,0.011331,0.000581,0.005811,0.006973,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
car_eval_4_LightGBM+optuna_all,0.029395,0.047421,0.033833,0.030505,0.026345,0.018580,0.013866,0.050749,0.019412,0.013866,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
yeast_me2_LightGBM+optuna_all,0.155823,0.131766,0.159103,0.094587,0.000000,0.000000,0.091853,0.039913,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
aggregate_df.to_csv(ROOT_DIR + "/logs/results.csv", index=False)

In [42]:
importances_df.to_csv(ROOT_DIR + "/logs/results_importances.csv", index=False)

In [50]:
kdd99_records = fetch_all_records({"dataset.name": "kdd99"})

In [51]:
kdd99_records = list(kdd99_records)

In [52]:
len(kdd99_records)

119

In [53]:
kdd99_df = pd.DataFrame(kdd99_records)

In [54]:
aggregate_kdd99_df = pd.DataFrame([
    {
        "id": result['_id'],
        "version": result['version'],
        "datetime": result['datetime'],
        "elapsed_time": result['elapsed_time'],
        "dataset": result['dataset']['name'] + "_d" if result['dataset'].get("dropped") == True else result['dataset']['name'],
        "ae_used_data": result['dataset']['ae_used_data'],
        "feature_num": result['dataset']['total_feature'],
        "model": result['model_name'],
        "layers": result['encoder_param']['layers'],
        **{k: result['result'][k]["f1-score"] for k in result['result']},
        }
    for result in kdd99_records
])
aggregate_kdd99_df.head()

,id,version,datetime,elapsed_time,dataset,ae_used_data,feature_num,model,layers,normal,dos,probe,r2l,u2r,macro avg
0,6562588b28c2769e56b4cafc,1.2.5,2023-11-25 20:26:50.711,0:28:45.259836,kdd99,normal,43,LightGBM,"[20, 10, 5]",0.8303,0.8589,0.6371,0.5059,0.4686,0.6602
1,656258c5f3710409d538ff29,1.2.5,2023-11-25 20:27:49.048,0:29:43.537391,kdd99_d,normal,30,LightGBM,"[20, 10, 5]",0.7938,0.7163,0.6123,0.3091,0.0280,0.4919
2,65625912307c3e6f503ac32b,1.2.5,2023-11-25 20:29:06.402,0:23:09.550778,kdd99_d,all,25,LightGBM,[],0.8412,0.8586,0.8144,0.4980,0.4529,0.6930
3,6562599a2cdf278a82dd9bd7,1.2.5,2023-11-25 20:31:22.349,0:25:23.143118,kdd99,all,38,LightGBM,[],0.9930,0.9992,0.8928,0.6693,0.4554,0.8019
4,65625d89f3710409d538ff2b,1.2.5,2023-11-25 20:48:08.590,0:20:17.494237,kdd99_d,u2r,30,LightGBM,"[20, 10, 5]",0.8362,0.8568,0.7809,0.1587,0.2638,0.5793


In [59]:
# 特定のカラムのみ抽出
aggregate_kdd99_df[["dataset", "version", "ae_used_data", "feature_num", "model", "layers", "normal", "dos", "probe", "r2l", "u2r", "macro avg"]].to_csv(ROOT_DIR + "/logs/filtered_kdd99.csv", index=False)

In [72]:
aggregate_df = pd.DataFrame([
    {
        "id": result['_id'],
        "version": result['version'],
        "datetime": result['datetime'],
        "elapsed_time": result['elapsed_time'],
        "dataset": result['dataset']['name'],
        "feature_num": result['dataset']['total_feature'],
        "model": result['model_name'],
        "layers": result['encoder_param']['layers'],
        **{k: result['result'][k]["f1-score"] for k in result['result']},
        }
    for result in results if result['dataset']['name'] != "kdd99" \
        and result['dataset']['name'] != "creditcardfraud" \
            and result['dataset']['ae_used_data'] == "all"\

])

In [79]:
aggregate_df.to_csv(ROOT_DIR + "/logs/filtered_results.csv", index=False)